In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import torch
import torchvision
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader, Dataset
import os
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [0]:
# transforms = transforms.Compose([
#         transforms.Resize(256),
#         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], 
#                              [0.229, 0.224, 0.225])
#     ])

train_transforms = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
])

train_set = datasets.ImageFolder("/content/drive/My Drive/Skin Diseases Classification/train", train_transforms)
test_set   = datasets.ImageFolder("/content/drive/My Drive/Skin Diseases Classification/test", test_transforms)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=10,
                                       shuffle=True, num_workers=4)

test_loader = torch.utils.data.DataLoader(test_set, batch_size=10,  
                                       shuffle=True, num_workers=4)

In [0]:
classes = train_set.classes
device = torch.device("cuda:0" if torch.cuda.is_available()
                               else "cpu")

In [5]:
print(len(classes))

12


In [6]:
model = models.resnet152(pretrained=True, progress=True)

# ## freeze the layers
# for param in model.parameters():
#    param.requires_grad = False

# # Modify the last layer
# number_features = model.classifier[6].in_features
# features = list(model.classifier.children())[:-1] # Remove last layer
# features.extend([torch.nn.Linear(number_features, len(classes))])
# model.classifier = torch.nn.Sequential(*features)

# model = model.to(device)

# print(model)

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.cache/torch/checkpoints/resnet152-b121ed2d.pth
100%|██████████| 230M/230M [00:00<00:00, 263MB/s]


In [0]:
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(classes))
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [8]:
for epoch in range(65):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    print(epoch+1, running_loss)
print('Finished Training')

1 2228.1398809552193
2 1942.9198725819588
3 1791.9094329476357
4 1672.8208201229572
5 1589.0750830471516
6 1503.3320297896862
7 1443.7263098806143
8 1375.5171193778515
9 1323.5344216376543
10 1269.2076473087072
11 1207.8337228298187
12 1165.7247332185507
13 1129.805430598557
14 1089.2958146333694
15 1056.6749850586057
16 1044.1869249641895
17 958.347407899797
18 995.5385451205075
19 922.6056658029556
20 909.2339998930693
21 868.9978851824999
22 851.9180238619447
23 810.8286679312587
24 821.4925300497562
25 772.0877097919583
26 765.3516315259039
27 752.2673173099756
28 729.8595893215388
29 702.9602487571537
30 702.1986355148256
31 679.6455775983632
32 632.2648954410106
33 647.7719530556351
34 628.4846987905912
35 634.548499211669
36 622.7177089825273
37 601.5194120518863
38 592.5630157701671
39 585.5487768389285
40 550.7244002604857
41 549.2270880779251
42 549.2960792351514
43 548.8747247867286
44 520.3825135622174
45 516.3811749853194
46 509.68021234869957
47 500.95886472426355
48 491.

In [9]:
class_correct = list(0. for i in range(len(classes)))
class_total = list(0. for i in range(len(classes)))
with torch.no_grad():
    for i, data in enumerate(test_loader, 0):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(4):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
for i in range(len(classes)):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of Acne and Rosacea Photos : 74 %
Accuracy of Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions : 75 %
Accuracy of Atopic Dermatitis Photos : 71 %
Accuracy of Eczema Photos : 61 %
Accuracy of Light Diseases and Disorders of Pigmentation : 59 %
Accuracy of Lupus and other Connective Tissue diseases : 55 %
Accuracy of Nail Fungus and other Nail Disease : 87 %
Accuracy of Psoriasis pictures Lichen Planus and related diseases : 69 %
Accuracy of Seborrheic Keratoses and other Benign Tumors : 79 %
Accuracy of Systemic Disease : 58 %
Accuracy of Tinea Ringworm Candidiasis and other Fungal Infections : 74 %
Accuracy of Warts Molluscum and other Viral Infections : 79 %


In [0]:
# mobile model
example = torch.rand(1, 3, 224, 224)
traced_script_module = torch.jit.trace(model.cpu(), example)
traced_script_module.save("/content/drive/My Drive/Skin Diseases Classification/resnet152-65epoch.pt")